In [12]:
import gym
import numpy as np
import pandas as pd
from itertools import count
import pickle
from collections import namedtuple
import h2o
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from collections import OrderedDict
import warnings
import random
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')
h2o.init()
h2o.no_progress()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,35 mins 50 secs
H2O_cluster_timezone:,Asia/Taipei
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,2 months and 29 days
H2O_cluster_name:,H2O_from_python_chris_lo_w0o0zk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.981 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [2]:
# 預先在雲端硬碟中上傳 new_data.csv
injection_data_path = '../data/16539338267026_new_data.csv'

# 輸出檔案至雲端硬碟
scalar_path = '../data/scalar-20220803-factor.pickle'
save_model_path = '../model/h2o_injection_model-20220803-factor.model/StackedEnsemble_AllModels_1_AutoML_1_20250626_101532'
a2c_model_path = '../model/a2c-20220803.model'

In [3]:
def preproc_to_raw_data(data_preproc_):
  # onehot 後的data 轉回raw data
  df_ = pd.DataFrame()
  for c in X_features:
    dt = one_hot_enc[c].inverse_transform(data_preproc_.loc[:, data_preproc_.columns.str.startswith(c)])
    dt = label_enc[c].inverse_transform(dt)
    df_[c] = dt

  df_[y_feature] = data_preproc_[y_feature].apply(lambda x:10**x)
  return df_

def raw_data_to_preproc(data):
  # rawdata  轉onehot data
  data_ = data.copy()
  df_ = pd.DataFrame()
  for c in X_features:
    data_[c] = label_enc[c].transform(data_[c])
  for c in X_features:
    dt = pd.DataFrame(one_hot_enc[c].transform(data_[[c]]).toarray())
    dt = dt.add_prefix('%s_'%c)
    dt = dt.astype(int)
    df_ = pd.concat([df_.reset_index(drop=True),dt],axis=1)
  df_ = df_.astype(int)
  df_[y_feature] = np.log10(data_[y_feature].values)
  return df_

def mask_action(data):
  # 需要遮蔽的 action, 因為有機會輸出當前已經存在的參數
  def flatten(t):
    return [item for sublist in t for item in sublist]
  data_ = data.copy()
  mask_loc = []
  data_ = data_[X_features]
  for c in X_features:
    idx = action_table[(action_table['params']==c)&(action_table['value_']==data_[c].values[-1])].index
    mask_loc.append(idx)
  return flatten(mask_loc)


In [13]:
injection_model = h2o.load_model(save_model_path)
with open(scalar_path, 'rb') as f:
  label_enc = pickle.load(f)['label']
with open(scalar_path, 'rb') as f:
  one_hot_enc = pickle.load(f)['one_hot']

data = pd.read_csv(injection_data_path)
from scipy import stats
import math
data_ = data.copy().dropna()#[(np.abs(stats.zscore(data)) < 3).all(axis=1)]
y_feature = '重量'
X_features = ['射出一速_最高壓力','射出一速_速度','射出二速_最高壓力','射出二速_速度','儲料位置','一段保壓_速度']
for i in data_.columns:
  if i == '一段保壓_速度':
    data_[i] = data_[i].apply(lambda v: math.floor(v * 10) / 10.0)
  else:
    data_[i] = data_[i].apply(lambda v: math.floor(v))



data_preproc = raw_data_to_preproc(data_)
data_preproc.head(5)


,射出一速_最高壓力_0,射出一速_最高壓力_1,射出一速_最高壓力_2,射出一速_最高壓力_3,射出一速_最高壓力_4,射出一速_最高壓力_5,射出一速_最高壓力_6,射出一速_最高壓力_7,射出一速_最高壓力_8,射出一速_最高壓力_9,...,一段保壓_速度_8,一段保壓_速度_9,一段保壓_速度_10,一段保壓_速度_11,一段保壓_速度_12,一段保壓_速度_13,一段保壓_速度_14,一段保壓_速度_15,一段保壓_速度_16,重量
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2.093422
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.093422
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.093422
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.093422
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.093422


In [14]:
action_dict = {}
for c in X_features:
  values = data_[c].unique().tolist()
  values.sort()
  action_dict[c] = values
  print(f'{c} : 最大值:{np.max(values)} 最小值:{np.min(values)}')

射出一速_最高壓力 : 最大值:136 最小值:77
射出一速_速度 : 最大值:128 最小值:0
射出二速_最高壓力 : 最大值:143 最小值:117
射出二速_速度 : 最大值:115 最小值:0
儲料位置 : 最大值:103 最小值:92
一段保壓_速度 : 最大值:1.6 最小值:0.0


In [15]:
# 將所有的action 定義成一張table,透過index查詢動作

action_dict = OrderedDict(action_dict)
action_table = pd.DataFrame()
params = []
true_value = []
for k in action_dict.keys():
  for v in action_dict[k]:
    params.append(k)
    true_value.append(v)
action_table['params'] = params
action_table['value_'] = true_value
action_table

,params,value_
0,射出一速_最高壓力,77.0
1,射出一速_最高壓力,90.0
2,射出一速_最高壓力,91.0
3,射出一速_最高壓力,92.0
4,射出一速_最高壓力,93.0
...,...,...
186,一段保壓_速度,1.2
187,一段保壓_速度,1.3
188,一段保壓_速度,1.4
189,一段保壓_速度,1.5


In [16]:
# 可調參數
# reward discount gamma
GAMMA = 0.99
# 每訓練幾步 print 出結果
LOG_INTERVAL = 10

# State 紀錄 5 次
OBSERVATION_N = 5
# State column size 98
OBSERVATION_COLUMNS = data_preproc.shape[1]

# 可以輸出的動做個數 = action_table.shape[0]
ACTION_N = action_table.shape[0]
# reward alpha,beta
REWARD_ALPHA = 0.5
REWARD_BETA = 0.5
# 目標要達成的strength
TARGET_Y1 = 140

# Buffer size
BUFFER_N = 20

# done reward thres
DONE_THRES = -2.5

# max run episode
EPISODE_MAX = 5000


init_params = {'射出一速_最高壓力':[100],
 '射出一速_速度':[64],
 '射出二速_最高壓力':[120],
 '射出二速_速度':[50],
 '儲料位置':[93],
 '一段保壓_速度':[0.3],
 '重量':[124]}
init_params = pd.DataFrame.from_dict(init_params)
init_params

,射出一速_最高壓力,射出一速_速度,射出二速_最高壓力,射出二速_速度,儲料位置,一段保壓_速度,重量
0,100,64,120,50,93,0.3,124


In [17]:
def action_from_model(action):
  # action space = ACTION_N = [0.3,0.1,0.2,....]
  act_index = action #np.argmax(action)
  change_params_name = action_table.iloc[act_index]['params']
  change_params_value = action_table.iloc[act_index]['value_']
  return (change_params_name,change_params_value)


def change_params(data,param_name,value):

  data[param_name] = value
  return data

In [49]:
#from pandas._libs.tslibs.timedeltas import delta_to_nanoseconds
# injection 環境
class InjecttionEnv(gym.Env):
    
  def __init__(self,injection_model):
    self.action_space = gym.spaces.Discrete(ACTION_N)
    self.observation_space = gym.spaces.Box(low=0,high=3000,shape=(OBSERVATION_COLUMNS*OBSERVATION_N,))
    self.injection_model = injection_model
    self.current_state = None
    self.current_state_pandas = None
    self.times = 0
    self.y = y_feature
    self.x_names = X_features
      
  def step(self, action):
    self.times += 1
    done = False
    #mask_loc = mask_action(self.current_state_pandas.tail(1))
    # action[mask_loc] = -np.Inf
    params_name,change_value = action_from_model(action)
    current_state_df = self.current_state_pandas.tail(1).copy()
    current_state_df = change_params(current_state_df,params_name,change_value)
    #X = raw_data_to_preproc(current_state_df)[self.x_names]
    X = current_state_df.copy()
    for c in X_features:
      X[c] = label_enc[c].transform(X[[c]])
    X = h2o.H2OFrame(X)
    for c in X_features:
      X[c] = X[c].asfactor()

    strength = self.injection_model.predict(X).as_data_frame()['predict']

    current_state_df[self.y] = strength.values[0]
    
    #old code...
    #self.current_state_pandas = self.current_state_pandas.append(current_state_df)
    self.current_state_pandas = pd.concat([self.current_state_pandas,current_state_df],ignore_index=True)
    self.check_state_len()
    next_state = raw_data_to_preproc(self.current_state_pandas)
    # reward 定義 當前 strength 和 target strength 的差值
    delta = abs(self.current_state_pandas.tail(1)[y_feature].values[0] - TARGET_Y1)
    reward = -delta#(REWARD_ALPHA*delta+REWARD_BETA*self.times**2)


    mask_loc = mask_action(self.current_state_pandas.tail(1))
    # 當reward大於 DONE_THRES  結束
    if (reward > DONE_THRES) or self.times>10:
      done = True
    return self.state_warmup(next_state.values.flatten()), reward,mask_loc, done,None

  def check_state_len(self):
    if len(self.current_state_pandas) > OBSERVATION_N:
      self.current_state_pandas = self.current_state_pandas.iloc[1:]

  def state_warmup(self,state):
    k = OBSERVATION_COLUMNS*OBSERVATION_N
    n = state.shape[0]
    if n < k:
      zeros = np.zeros(shape=k)
      zeros[:n] = state
      return zeros
    else:
      return state

  def reset(self):
      # 以injection_process_old.csv 第一筆當作初始state
      df = init_params#preproc_to_raw_data(data_preproc.head(1))
      state = raw_data_to_preproc(df)
      self.current_state_pandas = df
      self.current_state = state
      self.times = 0
      return self.state_warmup(state.values.flatten()),mask_action(df)

In [50]:
env = InjecttionEnv(injection_model)
env.reset()

(array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [51]:
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

In [52]:
ACTIONS = action_table.index.tolist()

class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(OBSERVATION_COLUMNS*OBSERVATION_N, 512)
        # actor's layer
        self.action_head = nn.Linear(512, ACTION_N)

        # critic's layer
        self.value_head = nn.Linear(512, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        #print(self.affine1(x))
        x = F.relu(self.affine1(x))
        #print(x)
        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)
        m = torch.ones([ACTION_N,])

        #m[[mask,]] = 0.0
        action_prob = action_prob*m
        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t
        return action_prob, state_values


def random_action(state,mask):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    actions = [a for a in ACTIONS if a not in mask]
    action = random.choice(actions)
    action = torch.tensor(action)
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()


def select_action(state,mask):
    if np.random.uniform() < 0.0:
      return random_action(state,mask)
    else:
      state_ = torch.from_numpy(state).float()
      probs, state_value = model(state_)

        # create a categorical distribution over the list of probabilities of actions
      m = Categorical(probs)

        # and sample an action using the distribution
      action = m.sample()
      if action not in mask:
          # save to action buffer 將action ,value 存入 buffer
        model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
          #the action
        return action.item()
      else:
        return random_action(state,mask)

# def select_action(state,mask):

#       state_ = torch.from_numpy(state).float()
#       probs, state_value = model(state_,mask)

#         # create a categorical distribution over the list of probabilities of actions
#       m = Categorical(probs)

#         # and sample an action using the distribution
#       action = m.sample()
#       # save to action buffer 將action ,value 存入 buffer
#       model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
#           #the action
#       return action.item()


def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values


    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + GAMMA * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]

In [31]:
# 記錄所有參數結果
history = pd.DataFrame()

import wandb
# 紀錄reward 的工具
!wandb login --relogin

# 請登入帳號並輸入token


wandb: You can find your API key in your browser here: http://localhost:8080/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

In [ ]:
run = wandb.init(project='a2c-advice-20220803-135-v1.0.0')

In [53]:
def act_to_act_names(x):
  act_names = []
  for i in x:
    act_ = action_table.iloc[i]
    params = act_['params']
    value = act_['value_']
    act_name = f'{params}_{value}'
    act_names.append(act_name)
  return act_names

In [54]:
model = Policy()
eps = np.finfo(np.float32).eps.item()
optimizer = optim.Adam(model.parameters(), lr=3e-2)


In [57]:
best_mean_ep_reward = -TARGET_Y1
# run 5000 many episodes
state,mask = env.reset()
avg_rewards = []
saved_actions = []
for i_episode in range(EPISODE_MAX):
    #print(i_episode)
    # reset environment and episode reward
    #state,mask = env.reset()
    #history = history.append(env.current_state_pandas.iloc[-1])
    ep_reward = 0
    total_ep_reward = []

    # infinite loop while learning
    # 動作20步紀錄buffer 並訓練
    for t in range(1, BUFFER_N):

        # select action from policy
        action = select_action(state,mask)
        # take the action
        state, reward,mask, done,_ = env.step(action)
        saved_actions.append(action)
        #print(env.times)
        #history = history.append(env.current_state_pandas.tail(1))
        model.rewards.append(reward)
        #print(model.rewards)
        ep_reward += reward
        total_ep_reward.append(reward)

    last_state_params = env.current_state_pandas
    # perform backprop
    finish_episode()
    mean_ep_reward = np.mean(total_ep_reward)
    wandb.log({"Average reward": mean_ep_reward})
    avg_rewards.append(mean_ep_reward)
    if mean_ep_reward > best_mean_ep_reward:
      torch.save(model.state_dict(), a2c_model_path)
      best_mean_ep_reward = mean_ep_reward
    # log results
    if i_episode % LOG_INTERVAL  == 0:
        print('Episode {}\tAverage reward: {:.4f}\t reward history: '.format(
                  i_episode, mean_ep_reward), total_ep_reward)

    saved_actions_names = act_to_act_names(saved_actions)
    table = pd.DataFrame()
    table['action'] = saved_actions_names
    table['freq'] = 1
    table = table.groupby(['action'],as_index=False)[['freq']].sum()
    wandb_table = wandb.Table(data=table.values.tolist(), columns = ["action", "freq"])
    wandb.log({"action freq" : wandb.plot.bar(wandb_table, "action","freq", title="Actions Bar Chart")})
    wandb_table_last = wandb.Table(data=last_state_params.values.tolist(), columns = last_state_params.columns.values.tolist())
    run.log({"last_state_params": wandb_table_last})
    # check if we have "solved"
    if mean_ep_reward > DONE_THRES:
        print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(mean_ep_reward, t))
        break

Episode 0	Average reward: -15.9620	 reward history:  [-15.995526103858694, -15.995575147530104, -15.976735996789841, -15.971304789484037, -15.96505536490558, -15.971919552684895, -15.96505536490558, -15.93498887181508, -15.940295096411205, -16.02727390312296, -15.940295096411205, -15.946467764378284, -15.953150082333622, -15.946467764378284, -15.953477098185203, -15.946467764378284, -15.950130883338716, -15.948474863004918, -15.950130883338716]
Episode 10	Average reward: -13.9129	 reward history:  [-15.040784803403298, -15.041917930763205, -15.040784803403298, -15.041398746369481, -15.03950553180806, -15.03478448099564, -15.03950553180806, -15.028075518386359, -14.998432308293317, -15.052697623957485, -14.950216949593383, -14.908454194846499, -14.832622330022701, -14.753065515881175, -14.796835594066195, -14.753065515881175, -14.760450971538717, -5.1153507205070525, -5.11801041073619]
Episode 20	Average reward: -15.1819	 reward history:  [-15.430075937121856, -15.4279040467014, -15.428

H2OConnectionError: Unexpected HTTP error: ('Connection aborted.', BadStatusLine('GET /3/Jobs/$03017f00000132d4ffffffff$_a93093564cfa5965ff10bfaa59a64f38 HTTP/1.1\r\n'))

In [ ]:
last_state_params.to_csv('./best_params.csv',index=False)